In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [16]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

LENGTH = 1904
RECS = 1384 * 4
TRANSL = 252

X = np.zeros([RECS, LENGTH, 257], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(4):
    Xpart = np.load("datasets/clarin-long/data/clarin-rec-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-trans-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

5523 5536


In [ ]:
def mk_model(max_label_length):
    layer = feature_input = Input(shape = (None, NFEATS))
    for i in range(2):
        layer_0 = Lambda(K.expand_dims)(layer)
        layer_1 = Conv1D(8, [1,5], activation='linear', kernel_initializer=Orthogonal())(layer_0)
        layer_2 = LeakyReLU(0.01)(layer_1)
        layer_3 = Dropout(0.25)(layer_2)
        layer_4 = TimeDistributed(Flatten())(layer_3)
        layer_5 = Conv1D(512, 5, strides=2, padding = 'same', activation='linear', kernel_initializer=Orthogonal())(layer_4)
        layer_6 = LeakyReLU(0.01)(layer_5)
        layer = layer_7 = Dropout(0.25)(layer_6)
    for i in range(3):
        gating = Dense(512, activation='sigmoid')(layer)
        layer_10 = LSTM(512, return_sequences = True, kernel_initializer=Orthogonal())(layer)
        layer_11 = Dropout(0.25)(layer_10)
        layer = layer_12 = Lambda(lambda l: l[0] * l[1] + (1 - l[0]) * l[2])([gating, layer_11, layer])
    layer_15 = LSTM(NPHONES + 1, return_sequences = True, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

#override
def mk_model(max_label_length):
    layer = feature_input = Input(shape = (None, NFEATS))
    for i in range(1):
        layer_0 = Lambda(K.expand_dims)(layer)
        layer_1 = Conv2D(5, [1,5], activation='linear', kernel_initializer=Orthogonal())(layer_0)
        layer_2 = LeakyReLU(0.01)(layer_1)
        layer_3 = Dropout(0.25)(layer_2)
        layer_4 = TimeDistributed(Flatten())(layer_3)
        layer_5 = Conv1D(256, 5, strides=2, padding = 'same', activation='linear', kernel_initializer=Orthogonal())(layer_4)
        layer_6 = LeakyReLU(0.01)(layer_5)
        layer = layer_7 = Dropout(0.25)(layer_6)
    for i in range(1):
        gating = Dense(256, activation='sigmoid')(layer)
        layer_10 = LSTM(256, return_sequences = True, kernel_initializer=Orthogonal())(layer)
        layer_11 = Dropout(0.25)(layer_10)
        layer = layer_12 = Lambda(lambda l: l[0] * l[1] + (1 - l[0]) * l[2])([gating, layer_11, layer])
    layer_15 = LSTM(NPHONES + 1, return_sequences = True, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive


def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX, trainy, trainXl, trainyl], np.zeros(trainX.shape[0]), epochs = epochs, batch_size = 32)

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=1000) # at 300 it makes sensible predictions
    valid_data = make_data(*load_data(VALIDDATA))
    predictions = predict.predict(valid_data[0])
    DERs = validate(predictions, valid_data[2], valid_data[1], valid_data[3])
    print("Validation Digit Error Rate: {}".format(sum([x[0] * x[1] for x in DERs]) / sum([x[1] for x in DERs])))
    predict.save('cyfry.pred.h5')

[(5504, 1904, 257), (5504, 252), (5504, 1), (5504, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, None, 257)    0                                            
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, None, 257, 1) 0           input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, None, 253, 5) 30          lambda_9[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, None, 253, 5) 0           conv2d_5[0][0]                   
______________________________________________________

In [9]:
(X_lens >= Y_lens).mean()

0.9828605914302957

In [9]:
(X_lens == 0).mean()

0.0